In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import emoji
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline



def uppercase_ratio(text):
    total = len(text)
    if total == 0:
        return 0
    uppercase = sum(1 for c in text if c.isupper())
    return uppercase / total

def count_emojis(text):
    return sum(1 for c in text if c in emoji.EMOJI_DATA)

df = pd.read_csv("../data/data_for_CLUSTERIZATION.csv")
print(df.shape)
#Ищем дубликаты
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

#Удаляем ненужные столбцы
df = df.drop(["url", "video_id", "codec", "frame rate(est.)"], axis = 1)

#Работа с пропусками
#Удалим пустые строки если такие есть, и удалим строки с пустой целевой переменной
df = df.dropna(subset=['views'])
df = df.dropna(how='all')
print("Пропуски:\n", df.isnull().sum())
df['description'] = df['description'].fillna("[NO_DESC]")
df['hashtags'] = df['hashtags'].fillna("[NO_HASH]")
print("Пропуски в описании ", df['description'].isnull().sum())
print("Пропуски в тегах ", df['hashtags'].isnull().sum())
#df['hashtags_original'] = df['hashtags']
#df['hashtags_filled'] = df['hashtags'].fillna(df['title'].apply(title_to_hashtags))
#mask = df['hashtags_original'].isna()
#comparison = df.loc[mask, ['title', 'hashtags_original', 'hashtags_filled']].head(10)
#print(comparison)

#Работает  с разрешением
df['pixel_count'] = df['width'] * df['height']
df = df.drop(columns=['width', 'height'])

#Работаем с недопустимыми данными
numeric_cols = df.select_dtypes(include='number').columns
invalid_mask = (df[numeric_cols] == 0).any()
print("Столбцы с недопустимыми значениями:")
print(invalid_mask[invalid_mask].index.tolist())

df['bitrate'] = df['bitrate'].replace(0, np.nan)
df['bitrate(video)'] = df['bitrate(video)'].replace(0, np.nan)
df_clean = df.dropna(subset=['bitrate', 'bitrate(video)'])
print(f"Удалено строк: {len(df) - len(df_clean)}")

numeric_cols = df.select_dtypes(include='number').columns
invalid_mask = (df[numeric_cols] == 0).any()
print("Столбцы с недопустимыми значениями:")
print(invalid_mask[invalid_mask].index.tolist())



#Ящики с усами
plt.figure(figsize=(6, 4))
sns.boxplot(y=df['duration'])
plt.yscale('log')
plt.title('Распределение продолжительности (лог-шкала)')
plt.show()

plt.figure(figsize=(6, 4))
sns.boxplot(y=df['frame rate'])
plt.show()



plt.figure(figsize=(12, 7))
sns.boxplot(data=df, x='category', y='views')
plt.yscale('log')  # ←←← обязательно!
plt.xticks(rotation=45, ha='right')
plt.title('Связь между категорией и просмотрами', fontsize=14)
plt.ylabel('Просмотры (логарифмическая шкала)')
plt.xlabel('Категория')
plt.tight_layout()
plt.show()

#Гистограмма

plt.figure(figsize=(8, 4))
sns.histplot(df['views'], bins=50, log_scale=True)
plt.title('Распределение просмотров (логарифмическая шкала)')
plt.show()

views_for_log = df['views'].replace(0, 1)

plt.figure(figsize=(10, 6))
plt.hist(np.log10(views_for_log), bins=60, color='teal', edgecolor='black', alpha=0.7)
plt.title('Распределение просмотров (в логарифмическом масштабе)')
plt.xlabel('log₁₀(Просмотры)')
plt.ylabel('Количество видео')
plt.grid(True, linestyle='--', alpha=0.5)

# Подписи на оси X в "человеческом" виде
locs, labels = plt.xticks()
plt.xticks(locs, [f'{10**x:.0f}' for x in locs])
plt.show()

#Тепловая карта
columns = [
    'duration',
    'bitrate',
    'bitrate(video)',
    'frame rate',
    'likes',
    'comments',
    'pixel_count',
    'views'  # целевая переменная
]
df_subset = df[columns].copy()
corr_matrix = df_subset.corr(method='spearman')
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,          # показывать числа в ячейках
    fmt=".2f",           # формат: 2 знака после запятой
    cmap='coolwarm',     # цветовая палитра
    square=True,         # квадратные ячейки
    cbar_kws={"shrink": 0.8}
)
plt.title('Тепловая карта корреляций между признаками и просмотрами')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
#По карте видно что зависимость между битрейтами почти прямая => удаляем один из них
df = df.drop(['bitrate(video)'], axis = 1)
#Также видно что зависимость между числом пискеселей и битрейтом 0.88 его тоже можно удалить
df = df.drop(['bitrate'], axis = 1)

#Зависимсоть между названием и просмотрами
df['title_len'] = df['title'].str.len()
df['uppercase_ratio'] = df['title'].apply(uppercase_ratio)
df['emoji_count'] = df['title'].apply(count_emojis)
df['has_exclamation'] = df['title'].str.contains('!', na=False).astype(int)
columns_for_heatmap = [
    'title_len',
    'uppercase_ratio',
    'emoji_count',
    'has_exclamation',
    'views'
]
corr_matrix = df[columns_for_heatmap].corr(method='spearman')
plt.figure(figsize=(8, 6))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    square=True,
    cbar_kws={"shrink": 0.8}
)
plt.title('Тепловая карта: заголовок vs просмотры')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

df = df.drop(['title'], axis=1)
df = df.drop(['comments'], axis=1)
df = df.drop(['likes'], axis=1)
#Ноормализация
numerical_cols = ['duration', 'frame rate', 'pixel_count', 'title_len']
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

#Доработка оставшихся строк
df = pd.get_dummies(df, columns=['category'], prefix='category')
df['description'] = (df['description'] != "[NO_DESC]").astype(int)
df['hashtags'] = (df['hashtags'] != "[NO_HASH]").astype(int)


df.to_csv('../data/prepared_data_for_clusterization.csv', index=False)